In [ ]:
import pandas as pd
from datetime import datetime


<h1 id="1.-Data-Load">1. Data Load<a class="anchor-link" href="#1.-Data-Load">¶</a></h1>



<h2 id="1)미세먼지-데이터">1)미세먼지 데이터<a class="anchor-link" href="#1)미세먼지-데이터">¶</a></h2>


In [ ]:
#데이터 불러오기
df = pd.read_csv("./data.csv")

In [ ]:
#데이터에 결측치는 없다.
df.isnull().sum()

In [ ]:
df.head()

In [ ]:
#인덱스인 첫번째 칼럼 삭제
df.drop(df.columns[0], axis=1, inplace=True)

In [ ]:
#1484개의 데이터와 13개의 피쳐로 이루어짐
df.shape

In [ ]:
############### 날짜 처리

In [ ]:
#초 삭제
df['dataTime'][0][:-6]
df['dataTime'] = df['dataTime'].apply(lambda x: x[:-6])

In [ ]:
#날짜 데이터로 바꾸기
df['dataTime'] = df['dataTime'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d").date())

In [ ]:
#날짜별 평균으로 바꾸기
data_by_day = df.groupby(df['dataTime'])

In [ ]:
#총 64일의 데이터가 나왔다!
data_by_day = data_by_day.mean()

In [ ]:
#2019-01-01 ~ 2019-02-28 사이의 데이터만 얻기
data_by_day = data_by_day[3:62]

In [1]:
#데이터프레임의 미세먼지 = pm10Value


<h2 id="2)-평균기온,-최고기온">2) 평균기온, 최고기온<a class="anchor-link" href="#2)-평균기온,-최고기온">¶</a></h2>


In [ ]:
temp = pd.read_csv('./tempair.csv', encoding="cp1252")

In [ ]:
#컬럼명 설정
temp.columns = ['date', 'avr_temp', 'highest_temp']

In [ ]:
#문자를 날짜데이터로 변환하기
temp['date'] = temp['date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d").date())

In [ ]:
#날짜를 인덱스로 설정
temp = temp.set_index('date')


<h2 id="3)-네이버-키워드-검색">3) 네이버 키워드 검색<a class="anchor-link" href="#3)-네이버-키워드-검색">¶</a></h2>


In [ ]:
keyword = pd.read_csv("./datalab.csv")

In [ ]:
#상위 6개 필요없는 설명데이터 삭제
keyword.drop(keyword.index[:6], inplace=True)

In [ ]:
#컬럼명 설정
keyword.columns = ['date', 'click']

In [ ]:
#문자열을 날짜데이터로 바꾸기
keyword['date'] = keyword['date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d").date())

In [ ]:
#날짜를 인덱스로 설정
keyword = keyword.set_index('date')

In [ ]:
#클릭횟수를 실수형(숫자)으로 바꾸기
keyword['click'] = keyword['click'].apply(lambda x: float(x))


<h2 id="3)-트위터-크롤링-데이터">3) 트위터 크롤링 데이터<a class="anchor-link" href="#3)-트위터-크롤링-데이터">¶</a></h2>


In [ ]:
#from twitterscraper import logging
from twitterscraper import main
from twitterscraper import query
from twitterscraper import tweet
import pandas as pd
import codecs, json
from twitterscraper import query_tweets

In [ ]:
from __future__ import division
import random
import requests
import datetime as dt
import json
from functools import partial
from multiprocessing.pool import Pool
from twitterscraper.tweet import Tweet

In [ ]:
#트위터 크롤링
if __name__ == '__main__':
    list_of_tweets = query_tweets("마스크 공기", begindate = dt.date(2019,1,1) ,enddate=dt.date(2019,3,1), poolsize =50)

    #print the retrieved tweets to the screen:
    for tweet in query_tweets("마스크 공기", begindate = dt.date(2019,1,1) ,enddate=dt.date(2019,3,1), poolsize =50):
        print(tweet)

In [ ]:
#리스트로 받기
text = []
time = []

In [ ]:
for i in range(len(list_of_tweets)) :
    text += [{"text" : list_of_tweets[i].text}]
    time += [{"time" : list_of_tweets[i].timestamp}]

In [ ]:
#데이터 프레임으로 받기
text_df = pd.DataFrame(text)
time_df = pd.DataFrame(time)

tweet_list = pd.concat([text_df,time_df],axis=1)

In [ ]:
#시간순으로 정렬하기
tweet_list = tweet_list.sort_values('time')
#년-월-일만 받기
tweet_list['time'] = tweet_list['time'].apply(lambda x: datetime.strptime(str(x)[:10], "%Y-%m-%d").date())
#날짜별로 묶기
tweet_group = tweet_list.groupby(tweet_list.time)
#날짜별 트윗 개수
tweet_group = pd.DataFrame(tweet_group.count()['text'])


<h2 id="4)-데이터-합치기">4) 데이터 합치기<a class="anchor-link" href="#4)-데이터-합치기">¶</a></h2>


In [ ]:
# data_by_day.head()
# data_by_day[['pm10Value']].head()
# temp.head()
# temp[['avr_temp','highest_temp']].head()
# keyword.head()
# tweet_group.head()

In [ ]:
#합친 데이터
data = pd.concat([data_by_day[['pm10Value']], temp, keyword, tweet_group], axis=1)
#칼럼명 설정
data.columns = ['fine_dust','avr_temp','highest_temp','click','tweet']


<h1 id="2.-데이터-분석">2. 분석<a class="anchor-link" href="#2.-데이터-분석">¶</a></h1>


In [ ]:
#변수간의 상관분석 : 실제 미세먼지양과 트위터간의 상관관계가 네이버 클릭 횟수보다 컸다.
data.corr()

In [ ]:
from sklearn.linear_model import LinearRegression

x = data[['avr_temp',"highest_temp"]]
y1 = data['fine_dust']
y2 = data['click']
y3 = data['tweet']

In [ ]:
#실제 미세먼지 양으로 선형회귀한 결과
reg1 = LinearRegression().fit(x, y1)
reg1.score(x, y1) #결정계수

In [ ]:
#네이버 클릭횟수로 선형회귀한 결과
reg2 = LinearRegression().fit(x, y2)
reg2.score(x, y2) #결정계수

In [ ]:
#트위터로 선형회귀한 결과
reg3 = LinearRegression().fit(x, y3)
reg3.score(x, y3) #결정계수

In [ ]:
#기온정보로 미세먼지를 예측하는건 힘들다.